# Morhunenko Mykola Homework 3

# Toronto

In [ ]:
import timeit

import numpy as np
import osmnx as ox
import networkx as nx
import random
import pulp as pl

### Building osmnx "driving" graph for Toronto

In [ ]:
G_toronto_drive = ox.graph_from_place('Toronto, Canada', network_type='drive')

In [ ]:
print('Number of nodes: {}'.format(G_toronto_drive.number_of_nodes()))
print('Number of edges: {}'.format(G_toronto_drive.number_of_edges()))

### One-way vs Two-way roads distribution

In [ ]:
# Red color represents one-way roads and blue represents two-way roads
ec = ['r' if data['oneway'] else 'b' for u, v, key, data in G_toronto_drive.edges(keys=True, data=True)]
ox.plot_graph(G_toronto_drive, edge_color=ec, node_size=0);

### The 11th element is center (start and finish)

In [ ]:
points = np.random.choice(G_toronto_drive.nodes, 10)
points = np.append(points, ox.get_nearest_node(G_toronto_drive, (43.718551, -79.336411), method="haversine"))

In [ ]:
ox.plot_graph(G_toronto_drive, node_size=[50 if data in points else 0 for data in G_toronto_drive.nodes()])

### Distance matrix (distances and time)

In [ ]:
distance_routs_matrix = []
for i in points:
    for j in points:
        distance_routs_matrix.append(nx.shortest_path(G_toronto_drive, i, j, weight='length', method='dijkstra'))

In [ ]:
ox.plot_graph_routes(G_toronto_drive, distance_routs_matrix, route_linewidth=2, node_size = [200 if data in points else 0 for data in G_toronto_drive.nodes()])

In [ ]:
ox.speed.add_edge_speeds(G_toronto_drive, fallback=50)
ox.speed.add_edge_travel_times(G_toronto_drive)

In [ ]:
time_routs_matrix = []

for i in range(11):
    for j in range(11):
        time_routs_matrix.append(nx.shortest_path(G_toronto_drive, points[i], points[j], weight='time', method='dijkstra'))

In [ ]:
ox.plot_graph_routes(G_toronto_drive, time_routs_matrix, route_linewidth=2, node_size = [200 if data in points else 0 for data in G_toronto_drive.nodes()])

In [ ]:
distances = []
for el in time_routs_matrix:
    d = 0;
    for ed in ox.utils_graph.get_route_edge_attributes(G_toronto_drive, el):
        d += ed["length"]
    else:
        distances.append(d)

### Travelling salesman problem

In [ ]:
m = pl.LpProblem('TSP', pl.LpMinimize)

In [ ]:
len(distances)

In [ ]:
x = {}
for i in range(11):
    for j in range(11):
        lowerBound = 0
        upperBound = 1

        # Forbid loops
        if i == j:
            upperBound = 0
            # print i,i

        # Create the decision variable and First constraint
        x[i,j] = pl.LpVariable('x' + str(i) + '_' + str(j), lowerBound, upperBound, pl.LpBinary)
        
m += pl.lpSum([distances[i*11+j] * x[i,j] for i in range(11) for j in range(11)])

for i in range(11):
    m += pl.lpSum([x[i,j] for j in range(11)]) == 1

for j in range(11):
    m += pl.lpSum([x[i,j] for i in range(11)]) == 1

In [ ]:
u = []

for i in range(0, 11):
    u.append(pl.LpVariable('u_' + str(i), cat='Integer'))
    
for i in range(0, 11):
    for j in range(i+1, 11):
        m += pl.lpSum([u[i] - u[j] + 10*x[i,j]]) <= 10


In [ ]:
st = m.solve()

In [ ]:
print(pl.LpStatus[st])

In [ ]:
for i in range(11):
    for j in range(11):
#         print(pl.value(x[i,j]))
        if (pl.value(x[i,j]) is not None) and pl.value(x[i,j]) >0:
            print ("x_" + str(i) + ',' + str(j) + ' : ' + str( pl.value(x[i,j]) ))


##### It doesn't work yet but I don't know how to fix it, I spent on it more then two days =(